In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Normalize, ColorJitter, RandomHorizontalFlip, RandomRotation
from PIL import Image
import os
import numpy as np
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
import tifffile

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define class names
class_names = ["background", "film", "basket", "cardboard", "video_tape", "filament", "bag"]
num_classes = len(class_names)

# Define which classes to ignore in evaluation (only background)
ignore_in_eval = [True, False, False, False, False, False, False]

# Set random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
set_seed(42)

Using device: cuda


In [2]:
# Define dataset paths using relative paths
import os

# Set up base directory (assumes the script/notebook is in the Hackathon folder)
base_dir = "."

# Define dataset paths
train_img_dir = os.path.join(base_dir, "hyper", "train")
train_mask_dir = os.path.join(base_dir, "labels_hyper_lt", "train")
val_img_dir = os.path.join(base_dir, "hyper", "val")
val_mask_dir = os.path.join(base_dir, "labels_hyper_lt", "val")

# Define transformations
train_mask_transform = Compose([
    RandomHorizontalFlip(p=0.5),
    RandomRotation(degrees=20)
])

val_mask_transform = Compose([])  # No transformation for validation masks

In [3]:
class HyperspectralDataset(Dataset):
    def __init__(self, hyper_dir, mask_dir, image_transform=None, mask_transform=None):
        self.hyper_dir = hyper_dir
        self.mask_dir = mask_dir
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        self.num_bands = 33  # Default value, will be updated after loading first image
        
        # Check if directories exist
        if not os.path.exists(hyper_dir):
            print(f"Warning: Hyperspectral directory not found: {hyper_dir}")
            self.image_filenames = []
        else:
            # Get and check image filenames - include all common image extensions
            self.image_filenames = sorted([
                f for f in os.listdir(hyper_dir) 
                if any(f.lower().endswith(ext) for ext in ['.tif', '.tiff', '.TIF', '.TIFF'])
            ])
            print(f"Found {len(self.image_filenames)} hyperspectral images in {hyper_dir}")
        
        if not os.path.exists(mask_dir):
            print(f"Warning: Mask directory not found: {mask_dir}")
            self.mask_filenames = []
        else:
            # Get and check mask filenames
            self.mask_filenames = sorted([
                f for f in os.listdir(mask_dir) 
                if any(f.lower().endswith(ext) for ext in ['.png', '.jpg', '.jpeg', '.bmp', '.PNG', '.JPG', '.JPEG', '.BMP'])
            ])
            print(f"Found {len(self.mask_filenames)} masks in {mask_dir}")
        
        # Ensure same number of images and masks
        if len(self.image_filenames) != len(self.mask_filenames):
            print("Warning: Number of images and masks doesn't match!")
            if len(self.image_filenames) == 0 or len(self.mask_filenames) == 0:
                print("Critical: One of the directories is empty while the other has files!")
                if len(self.image_filenames) == 0:
                    print(f"Image directory ({hyper_dir}) appears to be empty or has no supported files")
                if len(self.mask_filenames) == 0:
                    print(f"Mask directory ({mask_dir}) appears to be empty or has no supported files")
            else:
                min_len = min(len(self.image_filenames), len(self.mask_filenames))
                self.image_filenames = self.image_filenames[:min_len]
                self.mask_filenames = self.mask_filenames[:min_len]
                print(f"Using the first {min_len} files from each directory")
        
        # Check first image to get band count
        if len(self.image_filenames) > 0:
            sample_path = os.path.join(hyper_dir, self.image_filenames[0])
            try:
                image = tifffile.imread(sample_path)
                if len(image.shape) == 3 and image.shape[-1] > 3:
                    self.num_bands = image.shape[-1]
                elif len(image.shape) == 3:
                    self.num_bands = image.shape[0]
                else:
                    print(f"Warning: Unexpected image shape {image.shape}, defaulting to 33 bands")
                print(f"Detected {self.num_bands} spectral bands")
            except Exception as e:
                print(f"Error loading sample image: {str(e)}")
                print(f"Defaulting to {self.num_bands} bands")
    
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.hyper_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx])
        
        try:
            image = tifffile.imread(img_path).astype(np.float32)
            image = torch.tensor(image, dtype=torch.float32) / 255.0
            
            if len(image.shape) == 3 and image.shape[-1] > 3:
                image = image.permute(2, 0, 1)
            
            mask = Image.open(mask_path).convert("L")
            mask_np = np.array(mask, dtype=np.uint8)
            
            _, height, width = image.shape
            if mask_np.shape[0] != height or mask_np.shape[1] != width:
                mask = mask.resize((width, height), Image.NEAREST)
                mask_np = np.array(mask, dtype=np.uint8)
            
            mask_tensor = torch.tensor(mask_np, dtype=torch.long)
            
            if self.image_transform:
                for i in range(image.shape[0]):
                    band = image[i]
                    band_min = band.min()
                    band_max = band.max()
                    if band_min != band_max:
                        image[i] = (band - band_min) / (band_max - band_min)
                    else:
                        image[i] = torch.zeros_like(band)
            
            if self.mask_transform:
                seed = torch.randint(0, 2**32, (1,)).item()
                torch.manual_seed(seed)
                random.seed(seed)
                mask_pil = Image.fromarray(mask_np)
                mask_pil = self.mask_transform(mask_pil)
                mask_tensor = torch.tensor(np.array(mask_pil), dtype=torch.long)
            
            return image, mask_tensor
        
        except Exception as e:
            print(f"Error loading data at index {idx}, paths: {img_path}, {mask_path}")
            print(f"Error details: {str(e)}")
            num_bands = getattr(self, 'num_bands', 33)
            dummy_image = torch.zeros((num_bands, 256, 256), dtype=torch.float)
            dummy_mask = torch.zeros((256, 256), dtype=torch.long)
            return dummy_image, dummy_mask


In [4]:
class SpectralAttention(nn.Module):
    def __init__(self, in_channels, reduction_ratio=8):
        super(SpectralAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        # Calculate reduced channels with a minimum to avoid extremely small values
        reduced_channels = max(in_channels // reduction_ratio, 1)
        
        self.fc = nn.Sequential(
            nn.Linear(in_channels, reduced_channels, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(reduced_channels, in_channels, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # Get dimensions
        b, c, _, _ = x.size()
        
        # Spatial pooling
        y = self.avg_pool(x)
        
        # Reshape for FC layers
        y = y.view(b, c)
        
        # Apply FC layers
        y = self.fc(y)
        
        # Reshape back to spatial form
        y = y.view(b, c, 1, 1)
        
        # Channel-wise multiplication (attention mechanism)
        return x * y.expand_as(x)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class PretrainedHyperspectralUNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(PretrainedHyperspectralUNet, self).__init__()
        
        # Spectral Attention to focus on important bands
        self.spectral_attention = SpectralAttention(in_channels)
        
        # Spectral reduction layer to map from hyperspectral (33 bands) to RGB (3 bands)
        self.spectral_reduction = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 3, kernel_size=1)
        )
        
        # Load pre-trained ResNet34 encoder
        self.encoder = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
        
        # Get ResNet layers for skip connections
        self.initial = nn.Sequential(
            self.encoder.conv1,  # 3 -> 64
            self.encoder.bn1,
            self.encoder.relu
        )
        self.maxpool = self.encoder.maxpool  # 64 -> 64
        self.encoder1 = self.encoder.layer1  # 64 -> 64
        self.encoder2 = self.encoder.layer2  # 64 -> 128
        self.encoder3 = self.encoder.layer3  # 128 -> 256
        self.encoder4 = self.encoder.layer4  # 256 -> 512
        
        # Decoder
        self.upconv4 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder4 = self._decoder_block(512, 256)
        
        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder3 = self._decoder_block(256, 128)
        
        self.upconv2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder2 = self._decoder_block(128, 64)
        
        self.upconv1 = nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2)
        self.decoder1 = self._decoder_block(128, 64)
        
        # Final convolution
        self.final_conv = nn.Conv2d(64, num_classes, kernel_size=1)
    
    def _decoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # Store input dimensions for later use
        _, _, h, w = x.shape
        
        # Apply Spectral Attention to enhance important bands
        x = self.spectral_attention(x)
        
        # Reduce spectral dimensions from hyperspectral to RGB
        x = self.spectral_reduction(x)
        
        # Encoder
        x1 = self.initial(x)  # 3 -> 64
        p1 = self.maxpool(x1)  # 64 -> 64 (with maxpool)
        
        e1 = self.encoder1(p1)  # 64 -> 64
        e2 = self.encoder2(e1)  # 64 -> 128
        e3 = self.encoder3(e2)  # 128 -> 256
        e4 = self.encoder4(e3)  # 256 -> 512
        
        # Decoder with skip connections
        d4 = self.upconv4(e4)  # 512 -> 256
        d4 = torch.cat([d4, e3], dim=1)  # 256 + 256 -> 512
        d4 = self.decoder4(d4)  # 512 -> 256
        
        d3 = self.upconv3(d4)  # 256 -> 128
        d3 = torch.cat([d3, e2], dim=1)  # 128 + 128 -> 256
        d3 = self.decoder3(d3)  # 256 -> 128
        
        d2 = self.upconv2(d3)  # 128 -> 64
        d2 = torch.cat([d2, e1], dim=1)  # 64 + 64 -> 128
        d2 = self.decoder2(d2)  # 128 -> 64
        
        d1 = self.upconv1(d2)  # 64 -> 64
        d1 = torch.cat([d1, x1], dim=1)  # 64 + 64 -> 128
        d1 = self.decoder1(d1)  # 128 -> 64
        
        # Final convolution
        out = self.final_conv(d1)  # 64 -> num_classes
        
        # Ensure output has same spatial dimensions as input (in case of dimension mismatch)
        if out.shape[2] != h or out.shape[3] != w:
            out = F.interpolate(out, size=(h, w), mode='bilinear', align_corners=True)
        
        return out


In [6]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, inputs, targets):
        inputs_soft = F.softmax(inputs, dim=1)
        targets_onehot = F.one_hot(targets, num_classes=inputs.shape[1]).permute(0, 3, 1, 2).float()
        intersection = torch.sum(inputs_soft * targets_onehot, dim=(0, 2, 3))
        union = torch.sum(inputs_soft + targets_onehot, dim=(0, 2, 3))
        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, weight=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(weight=weight)

    def forward(self, inputs, targets):
        log_probs = F.log_softmax(inputs, dim=1)
        probs = torch.exp(log_probs)
        loss = (1 - probs) ** self.gamma * self.ce(inputs, targets)
        return loss.mean()

In [7]:
def calculate_iou(preds, masks, num_classes):
    """Calculate IoU for each class and mean IoU."""
    intersection = torch.zeros(num_classes).to(device)
    union = torch.zeros(num_classes).to(device)

    # Count presence of each class for debugging
    class_in_pred = [False] * num_classes
    class_in_target = [False] * num_classes

    for cls in range(num_classes):
        if ignore_in_eval[cls]:
            continue  # Skip background class in IoU calculation

        pred_inds = (preds == cls)
        target_inds = (masks == cls)
        
        # Check if class exists in predictions and targets
        class_in_pred[cls] = pred_inds.any().item()
        class_in_target[cls] = target_inds.any().item()

        intersection[cls] += (pred_inds & target_inds).sum().float()
        union[cls] += (pred_inds | target_inds).sum().float()

    # Calculate IoU for each class
    iou_per_class = []
    for cls in range(num_classes):
        if ignore_in_eval[cls]:
            iou_per_class.append(float('nan'))  # Mark background class as NaN
        else:
            if union[cls] > 0:
                iou = intersection[cls] / union[cls]
                iou_per_class.append(iou.item())
            else:
                # If class is not present in either prediction or ground truth
                iou_per_class.append(float('nan'))

    # Compute mean IoU excluding background and classes not present
    valid_ious = [iou for iou in iou_per_class if not np.isnan(iou)]
    mean_iou = np.mean(valid_ious) if valid_ious else 0.0

    return mean_iou, iou_per_class, class_in_pred, class_in_target

In [8]:
# Example: If you want to ignore some classes in evaluation, define them here
# For instance, if you have num_classes=4 and you want to ignore none:
# ignore_in_eval = [False, False, False, False]
# Adjust based on your project needs
ignore_in_eval = []

# Example class names; adapt to your own data
# class_names = ['Background', 'Class1', 'Class2', 'Class3', ...]
class_names = []

def train_pretrained_hyperspectral_model():
    # Create datasets
    train_dataset = HyperspectralDataset(
        hyper_dir=train_img_dir,
        mask_dir=train_mask_dir,
        image_transform=True,
        mask_transform=train_mask_transform
    )

    val_dataset = HyperspectralDataset(
        hyper_dir=val_img_dir,
        mask_dir=val_mask_dir,
        image_transform=True,
        mask_transform=val_mask_transform
    )

    # Check if datasets are empty
    if len(train_dataset) == 0:
        raise ValueError(f"Training dataset is empty. Please check paths: {train_img_dir} and {train_mask_dir}")
    if len(val_dataset) == 0:
        print("Warning: Validation dataset is empty. Will only train without validation.")

    # Get number of bands
    num_bands = train_dataset.num_bands
    print(f"Using {num_bands} spectral bands for model")

    # Create data loaders
    batch_size = 8  # Adjust if running into memory issues
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = (
        DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
        if len(val_dataset) > 0
        else None
    )

    # If your arrays for ignoring classes or naming them are empty, fill them based on num_classes
    global ignore_in_eval, class_names
    if not ignore_in_eval or len(ignore_in_eval) != num_classes:
        ignore_in_eval = [False] * num_classes
    if not class_names or len(class_names) != num_classes:
        class_names = [f"Class {i}" for i in range(num_classes)]

    # Initialize model
    model = PretrainedHyperspectralUNet(in_channels=num_bands, num_classes=num_classes).to(device)
    print("Model initialized with pre-trained ResNet34 backbone")

    # Calculate class weights
    class_weights = torch.ones(num_classes).to(device)
    class_weights[0] = 0.1  # Example of lowering the weight for background class

    # Define loss function (combined Focal + Dice)
    criterion = lambda outputs, targets: 0.5 * FocalLoss(weight=class_weights)(outputs, targets) + 0.5 * DiceLoss()(outputs, targets)

    # Initialize optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', patience=5, factor=0.5, min_lr=1e-6
    )

    num_epochs = 50
    best_iou = 0.0
    history = {'train_loss': [], 'train_iou': [], 'val_loss': [], 'val_iou': [], 'lr': []}

    for epoch in range(num_epochs):
        # -------------------------------
        # Training Phase
        # -------------------------------
        model.train()
        total_loss = 0.0
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        progress_bar = tqdm(train_loader, desc="Training")
        for images, masks in progress_bar:
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        # Average training loss for the epoch
        avg_train_loss = total_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)
        history['lr'].append(optimizer.param_groups[0]['lr'])

        # Compute training IoU
        model.eval()
        train_preds, train_masks = [], []
        with torch.no_grad():
            for images, masks in train_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                preds = torch.argmax(outputs, dim=1)
                train_preds.append(preds)
                train_masks.append(masks)

        train_preds = torch.cat(train_preds, dim=0)
        train_masks = torch.cat(train_masks, dim=0)
        train_mean_iou, train_iou_per_class, _, _ = calculate_iou(train_preds, train_masks, num_classes)
        history['train_iou'].append(train_mean_iou)

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Mean IoU: {train_mean_iou:.4f}")
        print("Train IoU per Class:")
        for cls in range(num_classes):
            if not ignore_in_eval[cls]:
                class_iou = train_iou_per_class[cls]
                if not np.isnan(class_iou):
                    print(f"  {class_names[cls]}: {class_iou:.4f}")
                else:
                    print(f"  {class_names[cls]}: N/A (not present)")

        # -------------------------------
        # Validation Phase
        # -------------------------------
        if val_loader is not None:
            model.eval()
            val_loss = 0.0
            val_preds, val_masks_all = [], []

            with torch.no_grad():
                for images, masks in tqdm(val_loader, desc="Validation"):
                    images, masks = images.to(device), masks.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, masks)
                    val_loss += loss.item()

                    preds = torch.argmax(outputs, dim=1)
                    val_preds.append(preds)
                    val_masks_all.append(masks)

            avg_val_loss = val_loss / len(val_loader)
            history['val_loss'].append(avg_val_loss)

            val_preds = torch.cat(val_preds, dim=0)
            val_masks_all = torch.cat(val_masks_all, dim=0)
            val_mean_iou, val_iou_per_class, classes_in_pred, classes_in_gt = calculate_iou(
                val_preds, val_masks_all, num_classes
            )
            history['val_iou'].append(val_mean_iou)

            print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {avg_val_loss:.4f}, Val Mean IoU: {val_mean_iou:.4f}")
            print("Validation IoU per Class:")
            for cls in range(num_classes):
                if not ignore_in_eval[cls]:
                    class_iou = val_iou_per_class[cls]
                    if not np.isnan(class_iou):
                        print(f"  {class_names[cls]}: {class_iou:.4f}")
                    else:
                        print(f"  {class_names[cls]}: N/A (not present)")

            # Classes present in predictions and ground truth
            print("Classes present in predictions:", [class_names[i] for i in range(num_classes) if classes_in_pred[i]])
            print("Classes present in ground truth:", [class_names[i] for i in range(num_classes) if classes_in_gt[i]])

            # Update scheduler based on validation IoU
            scheduler.step(val_mean_iou)

            # Check for best model (based on validation IoU)
            if val_mean_iou > best_iou:
                print(f"New best IoU achieved: {val_mean_iou:.4f} (previous: {best_iou:.4f})")
                best_iou = val_mean_iou
                torch.save(model.state_dict(), "best_model.pth")
                print("Best model saved!")
        else:
            # If no validation set exists, we can still step the scheduler on training IoU (optional)
            scheduler.step(train_mean_iou)

        print("-" * 50)

    # -------------------------------
    # Final Evaluation
    # -------------------------------
    # Load best model and do a final evaluation
    if os.path.exists("best_model.pth"):
        print("Loading best model for final evaluation...")
        model.load_state_dict(torch.load("best_model.pth"))
    else:
        print("Warning: No best_model.pth found; using last epoch model.")

    print("Final Model Evaluation:")
    if train_loader is not None:
        evaluate_model(model, train_loader, "Training Set")
    if val_loader is not None:
        evaluate_model(model, val_loader, "Validation Set")

    print("Training and evaluation complete!")

In [9]:
def evaluate_model(model, loader, dataset_name):
    model.eval()
    
    # Define loss function
    criterion = lambda outputs, targets: 0.5 * FocalLoss()(outputs, targets) + 0.5 * DiceLoss()(outputs, targets)
    
    total_loss = 0.0
    all_preds, all_masks = [], []
    
    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, dim=1)
            all_preds.append(preds)
            all_masks.append(masks)
    
    all_preds = torch.cat(all_preds, dim=0)
    all_masks = torch.cat(all_masks, dim=0)
    mean_iou, iou_per_class, _, _ = calculate_iou(all_preds, all_masks, num_classes)
    
    total_loss /= len(loader.dataset)
    print(f"{dataset_name} Results:")
    print(f"Loss: {total_loss:.4f}")
    print(f"Mean IoU: {mean_iou:.4f}")
    print("IoU per Class:")
    for cls in range(num_classes):
        if not ignore_in_eval[cls]:
            class_iou = iou_per_class[cls]
            if not np.isnan(class_iou):
                print(f"  {class_names[cls]}: {class_iou:.4f}")
            else:
                print(f"  {class_names[cls]}: N/A (not present)")
    print("-" * 30)

In [10]:
if __name__ == "__main__":
    # Train the model
    model = train_pretrained_hyperspectral_model()
    
    # Create data loaders for evaluation
    try:
        train_dataset = HyperspectralDataset(
            hyper_dir=train_img_dir,
            mask_dir=train_mask_dir,
            image_transform=True,
            mask_transform=None
        )
        
        val_dataset = HyperspectralDataset(
            hyper_dir=val_img_dir,
            mask_dir=val_mask_dir,
            image_transform=True,
            mask_transform=None
        )
        
        # Check if datasets are empty
        if len(train_dataset) == 0:
            print("Warning: Training dataset is empty for evaluation. Skipping training evaluation.")
            train_loader = None
        else:
            batch_size = 8
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
        
        if len(val_dataset) == 0:
            print("Warning: Validation dataset is empty for evaluation. Skipping validation evaluation.")
            val_loader = None
        else:
            batch_size = 8
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
        
        # Final evaluation
        print("Final Model Evaluation:")
        if train_loader is not None:
            evaluate_model(model, train_loader, "Training Set")
        if val_loader is not None:
            evaluate_model(model, val_loader, "Validation Set")
    except Exception as e:
        print(f"Error during evaluation: {e}")
        print("Skipping final evaluation.")

Found 514 hyperspectral images in .\hyper\train
Found 514 masks in .\labels_hyper_lt\train
Detected 256 spectral bands
Found 167 hyperspectral images in .\hyper\val
Found 167 masks in .\labels_hyper_lt\val
Detected 256 spectral bands
Using 256 spectral bands for model
Model initialized with pre-trained ResNet34 backbone

Epoch 1/50


Training: 100%|██████████| 65/65 [03:52<00:00,  3.58s/it, loss=1.15]


Epoch [1/50], Train Loss: 1.1947, Train Mean IoU: 0.0907
Train IoU per Class:
  Class 0: 0.4308
  Class 1: 0.0735
  Class 2: 0.0082
  Class 3: 0.0095
  Class 4: 0.0152
  Class 5: 0.0004
  Class 6: 0.0975


Validation: 100%|██████████| 21/21 [01:16<00:00,  3.65s/it]


Epoch [1/50], Val Loss: 1.1651, Val Mean IoU: 0.0921
Validation IoU per Class:
  Class 0: 0.4249
  Class 1: 0.0748
  Class 2: 0.0204
  Class 3: 0.0106
  Class 4: 0.0118
  Class 5: 0.0007
  Class 6: 0.1014
Classes present in predictions: ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']
Classes present in ground truth: ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']
New best IoU achieved: 0.0921 (previous: 0.0000)
Best model saved!
--------------------------------------------------

Epoch 2/50


...

Epoch [16/50], Train Loss: 0.8535, Train Mean IoU: 0.1740
Train IoU per Class:
  Class 0: 0.5576
  Class 1: 0.2530
  Class 2: 0.1767
  Class 3: 0.0217
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.2090


Validation: 100%|██████████| 21/21 [01:50<00:00,  5.28s/it]


Epoch [16/50], Val Loss: 1.0102, Val Mean IoU: 0.1310
Validation IoU per Class:
  Class 0: 0.5605
  Class 1: 0.1273
  Class 2: 0.1124
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.1164
Classes present in predictions: ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 6']
Classes present in ground truth: ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']
--------------------------------------------------

Epoch 17/50


Training:  57%|█████▋    | 37/65 [03:15<02:27,  5.28s/it, loss=0.815]


KeyboardInterrupt: 